In [18]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import pandas_ta as ta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()

# feature engineering and scaling
def prep(dataset):
    scaler = StandardScaler()

    macro_path1 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\IRP DATASET  - Copy of US_macroeconomics.csv"
    macro_path2 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\^VIX.csv"
    macro_path3 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\CORESTICKM159SFRBATL.csv"
    macro_path4 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\FFR.csv"
    macro_path5 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GDP.csv"
    macro_path6 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VIX9D_History.csv"
    macro_path7 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GVZ_History.csv"
    macro_path8 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\OVX_History.csv"
    macro_path9 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VVIX_History.csv"
    macro_data1 = pd.read_csv(macro_path1)
    macro_data2 = pd.read_csv(macro_path2)
    macro_data3 = pd.read_csv(macro_path3)
    macro_data4 = pd.read_csv(macro_path4)
    macro_data5 = pd.read_csv(macro_path5)
    macro_data6 = pd.read_csv(macro_path6)
    macro_data7 = pd.read_csv(macro_path7)
    macro_data8 = pd.read_csv(macro_path8)
    macro_data9 = pd.read_csv(macro_path9)

    dataset['Date'] = pd.to_datetime(dataset['Date'], dayfirst=True)
    dataset.set_index('Date', inplace=True)
    dataset['Target'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

    macro_data1['date'] = pd.to_datetime(macro_data1['date'], dayfirst=True)
    macro_data1.set_index('date', inplace=True)

    macro_data2['Date'] = pd.to_datetime(macro_data2['Date'], dayfirst=True)
    macro_data2.set_index('Date', inplace=True)

    macro_data3['Date'] = pd.to_datetime(macro_data3['Date'], dayfirst=True)
    macro_data3.set_index('Date', inplace=True)

    macro_data4['Date'] = pd.to_datetime(macro_data4['Date'], dayfirst=True)
    macro_data4.set_index('Date', inplace=True)

    macro_data5['Date'] = pd.to_datetime(macro_data5['Date'], dayfirst=True)
    macro_data5.set_index('Date', inplace=True)

    macro_data7['Date'] = pd.to_datetime(macro_data7['Date'], dayfirst=False)
    macro_data7.set_index('Date', inplace=True)

    macro_data8['Date'] = pd.to_datetime(macro_data8['Date'], dayfirst=False)
    macro_data8.set_index('Date', inplace=True)

    macro_data9['Date'] = pd.to_datetime(macro_data9['Date'])
    macro_data9.set_index('Date', inplace=True)

    merge_data1 = dataset.join(macro_data1, how='left').fillna(method='ffill')
    merge_data2 = merge_data1.join(macro_data2, how='left').fillna(method='ffill')
    merge_data3 = merge_data2.join(macro_data3, how='left').fillna(method='ffill')
    merge_data4 = merge_data3.join(macro_data4, how='left').fillna(method='ffill')
    merge_data5 = merge_data4.join(macro_data5, how='left').fillna(method='ffill')
    merge_data7 = merge_data5.join(macro_data7, how='left').fillna(method='ffill')
    merge_data8 = merge_data7.join(macro_data8, how='left').fillna(method='ffill')
    merge_final = merge_data8.join(macro_data9, how='left').fillna(method='ffill')

    merge_final['RSI (14D)'] = ta.rsi(merge_final['Close'], length=14)
    merge_final['20 Day CCI'] = ta.cci(high=merge_final['High'], low=merge_final['Low'], 
                                    close=merge_final['Close'], length=20)
    merge_final['Williams %R'] = ta.willr(high=merge_final['High'], low=merge_final['Low'], 
                                        close=merge_final['Close'], length=14)
    merge_final['EMA (5D)'] = merge_final['Close'].ewm(span=5, adjust=False).mean()
    merge_final['MA50'] = merge_final['Close'].rolling(window=50).mean()

    final_dataset = merge_final.dropna()

    time_shifted = final_dataset.resample('M').asfreq().dropna().head(102)
    
    features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'FFR']
    
    time_shifted[features] = time_shifted[features].astype(float)
    time_shifted[features] = scaler.fit_transform(time_shifted[features])

    return time_shifted, scaler

# get dataset
file_path = filedialog.askopenfilename(parent=root, title="Select A File")
ticker = pd.read_csv(file_path)
ticker, scaler = prep(ticker)

ticker

,Open,High,Low,Close,Adj Close,Volume,Target,CPI,Mortgage_rate,Unemp_rate,...,FFR,GDP,GVZ,OVX,VVIX,RSI (14D),20 Day CCI,Williams %R,EMA (5D),MA50
Date,,,,,,,,,,,,,,,,,,,,,
2009-09-30,-1.3252,-1.3308,-1.3370,-1.3318,-1.3318,2.0522,0.0000,-1.7073,1.8940,1.4602,...,-0.6101,14448.8820,0.8027,0.4616,-1.5121,0.3230,0.1065,-0.4081,-1.3287,-1.3758
2009-11-30,-1.2902,-1.2915,-1.2903,-1.2868,-1.2868,-0.2479,1.0000,-1.6623,1.7104,1.5477,...,-0.5278,14651.2490,1.7548,0.2395,-0.8183,-0.1326,-0.0246,-0.7196,-1.2809,-1.3099
2009-12-31,-1.2489,-1.2562,-1.2569,-1.2640,-1.2640,-2.2366,1.0000,-1.6042,1.6721,1.5040,...,-0.6375,14651.2490,1.2395,-0.0290,-1.2639,-0.1068,0.3752,-0.0957,-1.2549,-1.2861
2010-03-31,-1.1964,-1.2018,-1.1972,-1.2005,-1.2005,0.3960,1.0000,-1.6038,1.7373,1.5040,...,-0.5827,14651.2490,0.1184,-0.3926,-1.0949,1.1194,0.4550,0.3499,-1.1979,-1.2560
2010-04-30,-1.1556,-1.1630,-1.1731,-1.1804,-1.1804,2.1063,1.0000,-1.6003,1.9645,1.5040,...,-0.4319,14980.1930,0.5773,-0.3576,0.5226,-0.5653,-0.5907,-1.6136,-1.1673,-1.2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30,2.3234,2.3084,2.3286,2.3181,2.3181,0.1954,1.0000,1.8205,-1.5834,-0.2015,...,-0.6375,23292.3620,-0.5982,0.0875,1.0507,0.6852,0.5269,0.0533,2.3245,2.1849
2021-06-30,2.4310,2.4295,2.4611,2.4541,2.4541,0.1573,1.0000,2.1138,-1.7313,-0.2452,...,-0.5964,23292.3620,-0.3184,-0.2215,0.8823,0.9597,1.2854,1.2001,2.4371,2.4213
2021-08-31,2.7091,2.6953,2.7281,2.7171,2.7171,0.1842,1.0000,2.1990,-1.9176,-0.4639,...,-0.6238,23828.9730,-0.4190,-0.1372,0.8201,0.9611,1.2234,1.0281,2.6989,2.6480


In [8]:
pd.options.display.float_format = '{:.4f}'.format

vif_ticker = pd.DataFrame()
features = ticker[['Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'MA50', 'FFR']]
vif_ticker["feature"] = features.columns

vif_ticker["VIF"] = [variance_inflation_factor(features.values, i)
                          for i in range(len(features.columns))]
vif_ticker

# corr_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
#                 'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
#                 'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'FFR', 'Target']

# corr_test = ticker[corr_columns]

# correlation_matrix = corr_test.corr()

# correlation_matrix['Target'].sort_values()

,feature,VIF
0,Close,255.1841
1,Volume,1.4793
2,CPI,59.3722
3,Mortgage_rate,5.6749
4,Unemp_rate,5.1628
5,disposable_income,11.2171
6,GVZ,3.0793
7,OVX,3.8599
8,VVIX,3.0174
9,RSI (14D),9.5485


In [20]:
# create LSTM model class
class LSTM_Model(nn.Module):
    def __init__(self, input_layer, hidden_layer, output_layer, num_layers=4, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.hidden_layer = hidden_layer
        self.num_layers = num_layers
        
        # LSTM layers with dropout
        self.lstm = nn.LSTM(input_layer, hidden_layer, num_layers, batch_first=True, dropout=dropout)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Linear layer
        self.linear_layer = nn.Linear(hidden_layer, output_layer)

    def forward(self, input_tensor):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        c0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        
        # Pass through LSTM layers
        out, _ = self.lstm(input_tensor, (h0, c0))
        
        # Apply dropout
        out = self.dropout(out[:, -1, :])
        
        # Pass through linear layer
        out = self.linear_layer(out)
        return out

# create sequences for input data and corresponding labels
def create_sequence(input_data, sequence_length):
    sequences = []
    for i in range(0, len(input_data) - sequence_length):
        sequence = input_data[i : i + sequence_length, :-1]
        label = input_data[i + sequence_length, -1]
        sequences.append((sequence, label))
    return sequences

# train the model with data provided
def trainer(model, train_data, loss_func, opt, epochs, fold, losses):
    model.train()
    for epoch in range(epochs):
        for sequence, labels in train_data:
            opt.zero_grad()
            sequence = sequence.clone().detach().float()
            labels = labels.clone().detach().float().view(-1, 1)
            
            y = model(sequence)
            loss = loss_func(y, labels)
            loss.backward()
            opt.step()

        if epoch % 25 == 1:
            print(f'Epoch {epoch} loss: {loss.item():.4f}')

            new_row = pd.DataFrame({'Fold': [fold+1], 'Epoch': [epoch], 'Loss': [loss.item()]})
            losses = pd.concat([losses, new_row], ignore_index=True)
            
    return losses

def predictor(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for sequence, _ in test_data:
            sequence = sequence.clone().detach().float()
            y = model(sequence)
            batch_predictions = torch.sigmoid(y)

            # Convert predictions to tensor if they are scalar or float
            if isinstance(batch_predictions, float) or batch_predictions.ndimension() == 0:
                batch_predictions = torch.tensor([batch_predictions])

            batch_predictions = torch.round(batch_predictions)

            predictions.extend(batch_predictions.squeeze().tolist())
    
    return predictions

# create sequences
sequence_length = 10
sequences = create_sequence(ticker[['Close', 'Volume', 'CPI', 'Mortgage_rate', 'disposable_income',
                                    'GVZ', 'OVX', 'VVIX', 'RSI (14D)', '20 Day CCI', 'Williams %R', 
                                    'MA50', 'Target']].values, sequence_length)

# cross-validation
tscv = TimeSeriesSplit(n_splits=10)
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []
losses = pd.DataFrame(columns=['Fold', 'Epoch', 'Loss'])

for fold, (train_index, test_index) in enumerate(tscv.split(sequences)):
    print(f'Fold {fold+1}')
    
    train_sequences = [sequences[i] for i in train_index]
    test_sequences = [sequences[i] for i in test_index]
    
    train_data = torch.utils.data.DataLoader(train_sequences, shuffle=True, batch_size=32)
    test_data = torch.utils.data.DataLoader(test_sequences, shuffle=False, batch_size=32)
    
    # initialize model
    model = LSTM_Model(input_layer=12, hidden_layer=50, output_layer=1)
    loss_func = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    # train model
    epochs = 100
    losses = trainer(model, train_data, loss_func, opt, epochs, fold, losses)
    
    # make predictions
    test_labels = [label for _, label in test_sequences]
    predictions = predictor(model, test_data)
    
    # calculate statistics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, zero_division=0)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    kappa = cohen_kappa_score(test_labels, predictions)
    
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)
    
    print(f'\nAccuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1: {f1:.4f}')
    print(f'Kappa: {kappa:.4f}\n')

# average scores across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_kappa = np.mean(kappas)

print(f'\nCross-Validation Results:')
print(f'Average Accuracy: {avg_accuracy:.4f}')
print(f'Average Precision: {avg_precision:.4f}')
print(f'Average Recall: {avg_recall:.4f}')
print(f'Average F1: {avg_f1:.4f}')
print(f'Average Kappa: {avg_kappa:.4f}')


Fold 1
Epoch 1 loss: 0.6955
Epoch 26 loss: 0.6888
Epoch 51 loss: 0.4911
Epoch 76 loss: 0.3468

Accuracy: 0.5000
Precision: 0.5000
Recall: 1.0000
F1: 0.6667
Kappa: 0.0000

Fold 2
Epoch 1 loss: 0.6952
Epoch 26 loss: 0.6947
Epoch 51 loss: 0.6410
Epoch 76 loss: 0.4228

Accuracy: 0.5000
Precision: 0.5000
Recall: 1.0000
F1: 0.6667
Kappa: 0.0000

Fold 3
Epoch 1 loss: 0.6934
Epoch 26 loss: 0.6816
Epoch 51 loss: 0.6063
Epoch 76 loss: 0.5150

Accuracy: 0.6250
Precision: 0.0000
Recall: 0.0000
F1: 0.0000
Kappa: -0.2000

Fold 4
Epoch 1 loss: 0.6628
Epoch 26 loss: 0.4905
Epoch 51 loss: 0.5619
Epoch 76 loss: 0.6505

Accuracy: 0.5000
Precision: 0.6000
Recall: 0.6000
F1: 0.6000
Kappa: -0.0667

Fold 5
Epoch 1 loss: 0.7028
Epoch 26 loss: 0.7028
Epoch 51 loss: 0.6099
Epoch 76 loss: 0.4876

Accuracy: 0.3750
Precision: 0.4286
Recall: 0.7500
F1: 0.5455
Kappa: -0.2500

Fold 6
Epoch 1 loss: 0.7019
Epoch 26 loss: 0.6313
Epoch 51 loss: 0.5630
Epoch 76 loss: 0.5850

Accuracy: 0.2500
Precision: 0.5000
Recall: 0.16

In [6]:
fall = (ticker.Target == 0).sum()
rise = (ticker.Target == 1).sum()

rise_per = (rise / (rise + fall)) * 100
fall_per = (fall / (rise + fall)) * 100

print(f'Number of days rising: {rise}')
print(f'Number of days falling: {fall}')

print(f'Rise % is: {rise_per:.2f}%')
print(f'Fall % is: {fall_per:.2f}%')

Number of days rising: 61
Number of days falling: 41
Rise % is: 59.80%
Fall % is: 40.20%


In [9]:
results = pd.DataFrame(columns=['Dataset', 'Average Accuracy', 'Average Precision', 
                                'Average Recall', 'Average F1', 'Average Kappa', 'Number of Rises', 
                                'Number of Falls', 'Rise %', 'Fall %'])

datatset_name = (file_path.split('/')[-1].split('.')[0])

metrics = {'Dataset': datatset_name ,'Average Accuracy': avg_accuracy, 
           'Average Precision': avg_precision, 'Average Recall': avg_recall, 
           'Average F1': avg_f1, 'Average Kappa': avg_kappa, 'Number of Rises': rise, 'Number of Falls': fall, 
           'Rise %': rise_per, 'Fall %': fall_per}

export_results = results._append(metrics, ignore_index=True)

export_results = pd.concat([export_results, losses], ignore_index=True)

export_results.set_index('Dataset', inplace=True)

export_results.to_csv(r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\IRP DATA\\" + datatset_name + '_MONTHLY_NEW.csv')

export_results

,Average Accuracy,Average Precision,Average Recall,Average F1,Average Kappa,Number of Rises,Number of Falls,Rise %,Fall %,Fold,Epoch,Loss
Dataset,,,,,,,,,,,,
^NYA,0.6375,0.6014,0.6733,0.6093,0.2017,61,41,59.8039,40.1961,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.6920
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,26,0.6845
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,51,0.4930
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,76,0.2777
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0.6913
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,26,0.6882
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,51,0.6349
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,76,0.4510
